# ProbGAN - Synthetic FX + Equity Forecasting Notebook
This notebook uses the `FXEQGenerator` from the ProbGAN repo to generate synthetic price paths based on FX and equity data.

In [5]:
!pip install torch easydict

# 📦 Environment Setup
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.autograd as autograd
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
from easydict import EasyDict
import torch.backends.cudnn as cudnn
from sklearn.preprocessing import StandardScaler
import joblib
from scipy.stats import skew, kurtosis, jarque_bera, ks_2samp



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 📈 Load and Preprocess Data
# Fit scaler on returns
raw_df = pd.read_csv("raw (FX + EQ).csv").dropna()
raw_returns = raw_df.pct_change().dropna().values

scaler = StandardScaler()
scaler.fit(raw_returns)


class FXEQGenerator(nn.Module):
    def __init__(self, z_size=100, out_dim=20*1):  # 1 asset
        super(FXEQGenerator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_size, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, out_dim)
        )

    def forward(self, z):
        return self.fc(z).view(-1, 20, 1)



class FXEQDiscriminator(nn.Module):
    def __init__(self, in_dim=20*1):
        super(FXEQDiscriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.fc(x.view(x.size(0), -1))


class FXEQDataset(Dataset):
    def __init__(self, csv_path, window_size=20, scaler=None, asset_index=0, split='train', split_ratio=0.65):
        self.data = pd.read_csv(csv_path).dropna()
        self.returns = self.data.pct_change().dropna().values[:, asset_index]  # ← Only one asset
        self.returns = self.returns.reshape(-1, 1)  # keep 2D for scaler

        # Split index
        split_point = int(len(self.returns) * split_ratio)
        if split == 'train':
            self.returns = self.returns[:split_point]
        elif split == 'test':
            self.returns = self.returns[split_point:]

        if scaler is not None:
            self.returns = scaler.transform(self.returns)

        self.returns = self.returns.squeeze()  # back to 1D
        self.window_size = window_size
        self.samples = [self.returns[i:i+window_size] for i in range(len(self.returns) - window_size)]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return torch.tensor(self.samples[idx], dtype=torch.float32).unsqueeze(1)  # shape: [window, 1]



def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def save_checkpoint(state, filename='checkpoint'):
    torch.save(state, filename + '.pth.tar')

def to_tensor(x):
    return x.clone().detach().to(device)


def parse():
    parser = argparse.ArgumentParser()
    """
    Hyper-parameter for Stochastic Gradient Hamiltonian Monte Carlo
    """
    parser.add_argument('--sghmc_alpha', default=0.01, type=int, dest='sghmc_alpha', help='number of generators')
    parser.add_argument('--g_noise_loss_lambda', default=3e-2, type=float, dest='g_noise_loss_lambda')
    parser.add_argument('--d_noise_loss_lambda', default=3e-2, type=float, dest='d_noise_loss_lambda')
    parser.add_argument('--d_hist_loss_lambda', default=1.0, type=float, dest='d_hist_loss_lambda')
    """
    GAN objectives
    NS: original GAN (Non-saturating version)
    MM: original GAN (Min-max version)
    W: Wasserstein GAN
    LS: Least-Square GAN
    """
    parser.add_argument('--gan_obj', default='NS', type=str, dest='gan_obj', help='[NS | MM | LS | W]')

    """
    Paths
    """
    parser.add_argument('--dataset', default='fxeq', type=str, dest='dataset',
                        help='dataset: [cifar10, stl10, imagenet]')
    parser.add_argument('--save_dir', default='none', type=str, dest='save_dir', help='save_path')

    return parser.parse_args()


def construct_model(args, config):
    '''
    :param args: Experiment Information
    :param config: Neural Network Architecture Configurations
    :return:
        G: generator structure
        D: discriminator structure
    '''
    D_unbound_output = args.gan_obj in ['W', 'LS']
    if config.image_size == 32:
        G = multi_generator_32(z_size=config.z_size, out_size=config.channel_size, ngf=config.ngf,
                               num_gens=config.num_gens).to(device)
        D = multi_discriminator_with_history(in_size=config.channel_size, ndf=config.ndf, num_discs=config.num_discs,
                                             unbound_output=D_unbound_output).to(device)
    if config.image_size == 48:
        G = multi_generator_48(z_size=config.z_size, out_size=config.channel_size, ngf=config.ngf,
                               num_gens=config.num_gens).to(device)
        D = multi_discriminator_48_with_history(in_size=config.channel_size, ndf=config.ndf, num_discs=config.num_discs,
                                                unbound_output=D_unbound_output).to(device)
    if args.dataset == 'fxeq':
        G = FXEQGenerator(z_size=config.z_size).to(device)
        D = FXEQDiscriminator().to(device)


    print('G #parameters: ', count_parameters(G))
    print('D #parameters: ', count_parameters(D))
    return G, D


def train_net(G, D, args, config):
    is_multi_g = hasattr(G, 'gs')
    is_multi_d = hasattr(D, 'ds') or hasattr(D, 'forward_by_hist')

    print(f"Using device: {device}")
    cudnn.benchmark = True

    noise_std = np.sqrt(2 * args.sghmc_alpha)
    G_noise_sampler = [get_sghmc_noise(g) for g in G.gs] if is_multi_g else [get_sghmc_noise(G)]
    D_noise_sampler = get_sghmc_noise(D)

    if args.save_dir == 'none':
        args.save_dir = './dump/train_{}_{}'.format(args.dataset, args.gan_obj)

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    train_loader = torch.utils.data.DataLoader(
        dataset=config.get_dataset(),
        batch_size=config.batch_size, shuffle=True,
        num_workers=config.workers, pin_memory=True)

    # setup loss function
    criterion_bce = nn.BCEWithLogitsLoss().to(device)
    criterion_mse = nn.MSELoss().to(device)
    if args.gan_obj == 'NS':
        phi_1 = lambda dreal, lreal, lfake: criterion_bce(dreal, lreal)
        phi_2 = lambda dfake, lreal, lfake: criterion_bce(dfake, lfake)
        phi_3 = lambda dfake, lreal, lfake: criterion_bce(dfake, lreal)
    elif args.gan_obj == 'MM':
        phi_1 = lambda dreal, lreal, lfake: criterion_bce(dreal, lreal)
        phi_2 = lambda dfake, lreal, lfake: criterion_bce(dfake, lfake)
        phi_3 = lambda dfake, lreal, lfake: - criterion_bce(dfake, lfake)
    elif args.gan_obj == 'LS':
        phi_1 = lambda dreal, lreal, lfake: criterion_mse(dreal, lreal)
        phi_2 = lambda dfake, lreal, lfake: criterion_mse(dfake, lfake)
        phi_3 = lambda dfake, lreal, lfake: criterion_mse(dfake, lreal)
    elif args.gan_obj == 'W':
        phi_1 = lambda dreal, lreal, lfake: - dreal.mean()
        phi_2 = lambda dfake, lreal, lfake: dfake.mean()
        phi_3 = lambda dfake, lreal, lfake: - dfake.mean()

    num_gs = len(getattr(G, 'gs', [G]))
    num_ds = getattr(D, 'n_ds', 1)

    # setup optimizer
    optimizerD = torch.optim.Adam(D.parameters(), lr=config.base_lr, betas=(config.beta1, 0.999))
    optimizerG = torch.optim.Adam(G.parameters(), lr=config.base_lr, betas=(config.beta1, 0.999))

    # setup some varibles
    batch_time = AverageMeter()
    data_time = AverageMeter()
    D_losses = AverageMeter()
    G_losses = AverageMeter()
    G_n_losses = AverageMeter()

    fixed_noise = torch.randn(100, config.z_size)
    with torch.no_grad():
        fixed_noise = Variable(fixed_noise.to(device))

    end = time.time()

    D.train()
    G.train()

    D_loss_list = []
    G_loss_list = []
    G_loss_by_hist_list = []
    score_list = []

    for epoch in range(config.epoches):
        i = 0
        for input in train_loader:

            '''
                Update D network:
            '''
            data_time.update(time.time() - end)

            batch_size = input.size(0)
            g_batch_size = config.g_batch_size
            assert g_batch_size >= batch_size

            input_var = Variable(input.to(device))

            # Train discriminator with real data
            label_real = torch.ones(max(batch_size, g_batch_size))
            label_real_var = Variable(label_real.to(device))

            D_real_result = D(input_var).squeeze()
            D_real_loss = phi_1(D_real_result, label_real_var[:batch_size], None)

            # Train discriminator with fake data
            label_fake = torch.zeros(g_batch_size)
            label_fake_var = Variable(label_fake.to(device))

            noise = torch.randn((g_batch_size, config.z_size)).to(device)
            noise_var = Variable(noise.to(device))
            G_result = G(noise_var)

            D_fake_result = D(G_result).squeeze()
            D_fake_loss = phi_2(D_fake_result, None, label_fake_var)

            # Back propagation
            D_train_loss = D_real_loss + D_fake_loss
            D_losses.update(D_train_loss.item())
            D_noise_loss = args.d_noise_loss_lambda * noise_loss(model=D, noise_sampler=D_noise_sampler,
                                                                 alpha=noise_std)
            D_train_loss += D_noise_loss

            if args.gan_obj == 'W':
                gradient_penalty = calc_gradient_penalty(D, input_var.data, G_result[:batch_size].data)
                D_train_loss += gradient_penalty

            D.zero_grad()
            D_train_loss.backward()
            optimizerD.step()

            '''
                Update G network:
            '''
            noise = torch.randn((g_batch_size, config.z_size)).to(device)
            noise_var = Variable(noise.to(device))
            G_result = G(noise_var)

            D_fake_result = D(G_result).squeeze()
            G_train_loss = phi_3(D_fake_result, label_real_var, label_fake_var)

            G_losses.update(G_train_loss.item())

            G_noise_loss = args.g_noise_loss_lambda * noise_loss(model=G, noise_sampler=G_noise_sampler[0], alpha=noise_std)

            G_train_loss += G_noise_loss

            if is_multi_d and hasattr(D, 'forward_by_hist'):
                D_fake_result_hist = D.forward_by_hist(G_result).squeeze()
                G_train_loss_by_hist = phi_3(D_fake_result_hist, label_real_var, label_fake_var)
                G_train_loss += G_train_loss_by_hist * args.d_hist_loss_lambda
                G_n_losses.update(G_train_loss_by_hist.item())


            # Back propagation
            D.zero_grad()
            G.zero_grad()
            G_train_loss.backward()
            optimizerG.step()

            batch_time.update(time.time() - end)
            end = time.time()

            
                # print(F.l1_loss(D.ds.weight.data, D.ds_hist_avg.weight.data))
            if is_multi_d and hasattr(D, 'update_hist') and i % 10 == 0:
                D.update_hist()


            if (i + 1) % config.display == 0:
                print_log_2(epoch + 1, config.epoches, i + 1, len(train_loader), config.base_lr,
                            config.display, batch_time, data_time, D_losses, G_losses, G_n_losses)
                batch_time.reset()
                data_time.reset()
            elif (i + 1) == len(train_loader):
                print_log_2(epoch + 1, config.epoches, i + 1, len(train_loader), config.base_lr,
                            (i + 1) % config.display, batch_time, data_time, D_losses, G_losses, G_n_losses)
                batch_time.reset()
                data_time.reset()

            i += 1

        D_loss_list.append(D_losses.avg)
        G_loss_list.append(G_losses.avg)
        G_loss_by_hist_list.append(G_n_losses.avg)

        D_losses.reset()
        G_losses.reset()
        G_n_losses.reset()

        if (epoch + 1) < config.dump_ep:
            plot_result(G, fixed_noise, config.image_size, epoch + 1, args.save_dir, is_gray=(config.channel_size == 1))
            plot_loss_my(D_loss_list, G_loss_list, G_loss_by_hist_list, epoch + 1, config.epoches, args.save_dir)

        if (epoch + 1) % config.dump_ep == 0:
            # plt the generate images and loss curve
            plot_result(G, fixed_noise, config.image_size, epoch + 1, args.save_dir, is_gray=(config.channel_size == 1))
            plot_loss_my(D_loss_list, G_loss_list, G_loss_by_hist_list, epoch + 1, config.epoches, args.save_dir)
            # save the D and G.
            save_checkpoint({'epoch': epoch, 'state_dict': D.state_dict(), },
                            os.path.join(args.save_dir, 'D_epoch_{}'.format(epoch)))
            save_checkpoint({'epoch': epoch, 'state_dict': G.state_dict(), },
                            os.path.join(args.save_dir, 'G_epoch_{}'.format(epoch)))
            torch.save(G.state_dict(), os.path.join(args.save_dir, 'gen_model.pt'))


        # monitoring gradient behavior & clip if needed
        tmp = grad_info(G.parameters())
        print('G grad l2-norm: {}, value max: {}'.format(tmp[0], tmp[1]))
        tmp = grad_info(D.parameters())
        print('D grad l2-norm: {}, value max: {}'.format(tmp[0], tmp[1]))

        nn.utils.clip_grad_norm_(parameters=G.parameters(), max_norm=100, norm_type=2)
        nn.utils.clip_grad_norm_(parameters=D.parameters(), max_norm=500, norm_type=2)

        if (epoch + 1) % config.dump_ep == 0:
            batch_size = 100
            total_size = 5000
            x = []
            for i in range(total_size // batch_size):
                noise = torch.randn((g_batch_size, config.z_size)).to(device)
                noise_var = Variable(noise.to(device))
                G_result = G(noise_var)
                x.append(G_result.detach().cpu().numpy())
            x = np.concatenate(x, axis=0)
            imgs = x
            # m, v = get_inception_score(images=imgs)
            m, v = 0, 0
            # fid = get_fid_score(images=imgs)
            fid = 0
            print('Epoch {} Inception Score: mean {:.6f} std {:.6f}'.format(epoch + 1, m, v))
            print('Epoch {} FID Score: {:.6f}'.format(epoch + 1, fid))

            score_list.append([epoch + 1, m, v, fid])
            plot_scores(score_list, args.save_dir)


def time_series_cv_split(data, window_size=20, train_size=0.65, val_window=100, step=50):
    n = len(data)
    split_point = int(n * train_size)
    splits = []

    for start in range(0, split_point - window_size - val_window + 1, step):
        train_end = start + window_size
        val_end = train_end + val_window

        if val_end > n:
            break

        train_data = data[start:train_end]
        val_data = data[train_end:val_end]
        splits.append((train_data.copy(), val_data.copy()))

    return splits



def noise_loss(model,
               noise_sampler,
               alpha):
    loss = 0
    for p, n in zip(model.parameters(), noise_sampler):
        n.normal_(mean=0, std=alpha)
        loss += torch.sum(p * n)
    return loss


def get_sghmc_noise(model):
    return [to_tensor(torch.zeros(p.size())) for p in model.parameters()]


# ======================================================================================================================
class AverageMeter(object):
    """ Computes ans stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.
        self.avg = 0.
        self.sum = 0.
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def grad_info(parameters):
    total_norm = 0
    total_abs_max = 0
    for p in parameters:
        if p.requires_grad:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
            vmax = p.grad.data.abs().max().item()
            if vmax > total_abs_max:
                total_abs_max = vmax
    total_norm = total_norm ** (1. / 2)
    return total_norm, total_abs_max



def calc_gradient_penalty(netD, real_data, fake_data, gp_lambda=10):
    # print real_data.size()
    assert len(real_data) == len(fake_data)
    alpha = torch.rand(len(real_data), 1, 1, 1)
    alpha = alpha.to(device)

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    interpolates = interpolates.to(device)
    interpolates = Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).to(device),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * gp_lambda
    return gradient_penalty


def calc_gradient_penalty_mgan(netD, real_data, fake_data, gp_lambda=10):
    # print real_data.size()
    assert len(real_data) == len(fake_data)
    alpha = torch.rand(len(real_data), 1, 1, 1)
    alpha = alpha.to(device)

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    interpolates = interpolates.to(device)
    interpolates = Variable(interpolates, requires_grad=True)

    disc_interpolates, _ = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).to(device),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * gp_lambda
    return gradient_penalty

from scipy.stats import skew, kurtosis, jarque_bera, ks_2samp

def plot_scores(score_list, save_dir):
    x = np.array(score_list)
    ep = x[:, 0]
    IS = x[:, 1]
    IS_std = x[:, 2]
    FID = x[:, 3]
    fig, ax = plt.subplots(1, 2, sharex='all', figsize=(12, 4.8))
    ax[0].set_xlabel('Epoch')
    ax[1].set_xlabel('Epoch')
    ax[0].set_ylabel('Inception Score')
    ax[1].set_ylabel('FID')
    ax[0].plot(ep, IS, 'r-', linewidth=3)
    ax[0].plot(ep, IS - IS_std, 'r--', linewidth=1)
    ax[0].plot(ep, IS + IS_std, 'r--', linewidth=1)
    ax[1].plot(ep, FID, 'r-', linewidth=3)

    plt.savefig(os.path.join(save_dir, 'score.png'))
    plt.close()
    np.save(os.path.join(save_dir, 'score.npy'), x)

def rolling_window_splits(data, window_size=20, test_size=100, step=10):
    splits = []
    for start in range(0, len(data) - window_size - test_size, step):
        train = data[start:start+window_size]
        test = data[start+window_size:start+window_size+test_size]
        splits.append((train, test))
    return splits



def plot_loss_my(d_loss, g_loss, g_loss_hist, num_epoch, epoches, save_dir):
    fig, ax = plt.subplots()
    ax.set_xlim(0, epoches + 1)
    ax.set_ylim(min(np.min(g_loss_hist), min(np.min(g_loss), np.min(d_loss))) - 0.1,
                max(np.max(g_loss), np.max(d_loss)) * 1.1)
    plt.xlabel('Epoch {}'.format(num_epoch))
    plt.ylabel('Loss')

    plt.plot([i for i in range(1, num_epoch + 1)], d_loss, label='Discriminator', color='red', linewidth=3)
    plt.plot([i for i in range(1, num_epoch + 1)], g_loss, label='Generator', color='mediumblue', linewidth=3,
             alpha=0.5)
    plt.plot([i for i in range(1, num_epoch + 1)], g_loss_hist, label='Generator - (hist)', color='green', linewidth=3,
             alpha=0.5)

    plt.legend()
    plt.savefig(os.path.join(save_dir, 'DCGAN_loss_epoch_{}.png'.format(num_epoch)))
    plt.close()


def plot_result(G, fixed_noise, image_size, num_epoch, save_dir, is_gray=False, n_series=None):
    G.eval()
    with torch.no_grad():
        generated = G(fixed_noise).cpu().numpy()  # shape: [batch_size, timesteps, features]

    if n_series is None:
        n_series = generated.shape[2]  # infer number of series

    fig, axes = plt.subplots(n_series, 1, figsize=(12, 2 * n_series), sharex=True)

    # Ensure axes is iterable even when n_series == 1
    if n_series == 1:
        axes = [axes]

    for i in range(n_series):
        series = generated[0, :, i]
        axes[i].plot(series)
        axes[i].set_ylabel(f'Series {i+1}')
        axes[i].grid(True)

    axes[-1].set_xlabel("Timestep")
    fig.suptitle(f"Generated Time Series Sample - Epoch {num_epoch}")
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f"time_series_epoch_{num_epoch}.png"))
    plt.close()




def print_log_2(epoch, epoches, iteration, iters, learning_rate,
                display, batch_time, data_time, D_losses, G_losses, G_n_losses):
    print('epoch: [{}/{}] iteration: [{}/{}]\t'
          'Learning rate: {}'.format(epoch, epoches, iteration, iters, learning_rate))
    print('Time {batch_time.sum:.3f}s / {0} iters, ({batch_time.avg:.3f})\t'
          'Data load {data_time.sum:.3f}s / {0} iters, ({data_time.avg:3f})\n'
          'Loss_D = {loss_D.val:.8f} (ave = {loss_D.avg:.8f})\n'
          'Loss_G = {loss_G.val:.8f} (ave = {loss_G.avg:.8f})\n'
          'Loss_GN = {loss_GN.val:.8f} (ave = {loss_GN.avg:.8f})\n'.format(
        display, batch_time=batch_time,
        data_time=data_time, loss_D=D_losses, loss_G=G_losses, loss_GN=G_n_losses))
    print(time.strftime('%Y-%m-%d %H:%M:%S '
                        '-----------------------------------------------------------------------------------------------------------------\n',
                        time.localtime()))


def generate_synthetic_forecast(G, forecast_horizon=300, n_samples=1, steps_per_pass=20):
    n_passes = forecast_horizon // steps_per_pass
    all_returns = []

    if scaler is None:
        raise ValueError("Scaler must be provided to inverse transform returns.")

    
    with torch.no_grad():
        for _ in range(n_passes):
            z = torch.randn(n_samples, 100).to(device)
            returns_block = G(z).cpu().numpy()
            all_returns.append(returns_block)

    returns = np.concatenate(all_returns, axis=1)

    # Inverse transform the generated synthetic returns
    returns_flat = returns.reshape(-1, returns.shape[2])  # shape: (timesteps * features)
    returns_unscaled = scaler.inverse_transform(returns_flat)
    returns = returns_unscaled.reshape(returns.shape)  # back to (1, timesteps, features)

    
    prices = []
    for i in range(returns.shape[2]):
        series = returns[0, :, i]
        series_prices = [LAST_KNOWN_PRICES[i]]
        for r in series:
            series_prices.append(series_prices[-1] * (1 + r))
        prices.append(series_prices[1:])
    
    df_synthetic = pd.DataFrame(np.array(prices).T, columns=df.columns)
    df_synthetic.to_csv("synthetic_prices.csv", index=False)
    return df_synthetic

def evaluate_synthetic_vs_real(real_returns, synthetic_returns):
    results = {
        "Mean_Real": np.mean(real_returns),
        "Mean_Synthetic": np.mean(synthetic_returns),
        "Std_Real": np.std(real_returns),
        "Std_Synthetic": np.std(synthetic_returns),
        "Skew_Real": skew(real_returns),
        "Skew_Synthetic": skew(synthetic_returns),
        "Kurt_Real": kurtosis(real_returns),
        "Kurt_Synthetic": kurtosis(synthetic_returns),
        "KS_Distance": ks_2samp(real_returns, synthetic_returns).statistic,
        "Jarque_Bera_Real": jarque_bera(real_returns)[0],
        "Jarque_Bera_Synthetic": jarque_bera(synthetic_returns)[0],
    }
    return pd.DataFrame([results])



config = EasyDict()
config.num_gens = 10
config.num_discs = 4
config.z_size = 100
config.channel_size = 1
config.ngf = 128
config.ndf = 128
config.batch_size = 64
config.g_batch_size = 128
config.base_lr = 0.0001
config.beta1 = 0.5
config.epoches = 200
config.dump_ep = 10
config.image_size = 1
config.get_dataset = lambda: FXEQDataset("raw (FX + EQ).csv", window_size=20, scaler=scaler)
config.display = 800
config.workers = 0


class Args:
    sghmc_alpha = 0.01
    g_noise_loss_lambda = 3e-2
    d_noise_loss_lambda = 3e-2
    d_hist_loss_lambda = 1.0
    gan_obj = "NS"
    dataset = "fxeq"
    save_dir = "fxeq_probgan_output"

args = Args()

joblib.dump(scaler, "fxeq_returns_scaler.pkl")




raw_df = pd.read_csv("raw (FX + EQ).csv").dropna()
raw_returns_all_assets = raw_df.pct_change().dropna().values
NUM_ASSETS = 12
all_metrics = []  # to hold metric results for all assets and folds

for asset_index in range(NUM_ASSETS):
    print(f"🔁 TSCV for Asset {asset_index}")
    returns = raw_returns_all_assets[:, asset_index].reshape(-1, 1)
    scaler = StandardScaler().fit(returns)
    returns_scaled = scaler.transform(returns).squeeze()

    splits = time_series_cv_split(returns_scaled, window_size=300, val_window=100, step=100)
    
    for fold, (train_seq, val_seq) in enumerate(splits):
        print(f"📦 Fold {fold + 1} | Train: {len(train_seq)}, Val: {len(val_seq)}")

        # Create Datasets for each fold
        class FoldDataset(Dataset):
            def __init__(self, series, window_size):
                self.series = series
                self.window_size = window_size
                self.samples = [series[i:i+window_size] for i in range(len(series) - window_size)]

            def __len__(self):
                return len(self.samples)

            def __getitem__(self, idx):
                return torch.tensor(self.samples[idx], dtype=torch.float32).unsqueeze(-1)

        config.get_dataset = lambda: FoldDataset(train_seq, window_size=20)

        # Re-init model per fold
        G, D = construct_model(args, config)

        # Train model
        train_net(G, D, args, config)

        # Save results per fold
        fold_dir = f"tscv_asset_{asset_index}/fold_{fold}"
        os.makedirs(fold_dir, exist_ok=True)
        torch.save(G.state_dict(), os.path.join(fold_dir, "G.pt"))
        torch.save(D.state_dict(), os.path.join(fold_dir, "D.pt"))
        # Evaluate synthetic vs real returns
        real_returns_flat = val_seq.flatten()
        real_prices = np.cumprod(1 + val_seq.flatten())
        G.eval()
        with torch.no_grad():
            z = torch.randn(1, config.z_size).to(device)
            synthetic_returns = G(z).cpu().numpy().flatten()
        synt_prices = [real_prices[0]]  # start from same point
        for r in scaler.inverse_transform(synthetic_returns.reshape(-1, 1)).flatten():
            synt_prices.append(synt_prices[-1] * (1 + r))
        synt_prices = np.array(synt_prices[1:])
        
        metrics_df = evaluate_synthetic_vs_real(real_prices, synt_prices)
        metrics_df["Asset"] = asset_index
        metrics_df["Fold"] = fold
        all_metrics.append(metrics_df)




results_df = pd.concat(all_metrics, ignore_index=True)
results_df.to_csv("price_level_evaluation_metrics.csv", index=False)
print("📄 All metrics saved to 'price_level_evaluation_metrics.csv'")

import matplotlib.pyplot as plt
import seaborn as sns

# 📦 Generate Boxplots for Each Asset
def plot_boxplots(real_all, synthetic_all, asset_names=None):
    """
    real_all, synthetic_all: list of arrays, each array is prices for one asset
    asset_names: optional list of asset names
    """
    num_assets = len(real_all)
    data = []
    for i in range(num_assets):
        real = real_all[i]
        synthetic = synthetic_all[i]
        asset = asset_names[i] if asset_names else f"Asset {i}"

        data.extend([
            {"Asset": asset, "Price": p, "Type": "Real"} for p in real
        ])
        data.extend([
            {"Asset": asset, "Price": p, "Type": "Synthetic"} for p in synthetic
        ])

    df = pd.DataFrame(data)
    plt.figure(figsize=(15, 6))
    sns.boxplot(data=df, x="Asset", y="Price", hue="Type", palette="Set2")
    plt.title("Boxplots of Real vs Synthetic Prices per Asset")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("price_boxplots_per_asset.png")
    plt.show()

# 🧱 Build arrays of real/synthetic prices across all assets (from LAST_KNOWN_PRICES)
real_all = []
synthetic_all = []
for asset_index in range(NUM_ASSETS):
    prices_real = np.cumprod(1 + raw_returns_all_assets[-300:, asset_index])
    G.eval()
    with torch.no_grad():
        z = torch.randn(1, config.z_size).to(device)
        synthetic = G(z).cpu().numpy().flatten()
        returns_unscaled = scaler.inverse_transform(synthetic.reshape(-1, 1)).flatten()
        prices_synth = [prices_real[0]]
        for r in returns_unscaled:
            prices_synth.append(prices_synth[-1] * (1 + r))
        prices_synth = prices_synth[1:]

    real_all.append(prices_real)
    synthetic_all.append(prices_synth)

plot_boxplots(real_all, synthetic_all)


# Plot heatmap of KS distance
import seaborn as sns
import matplotlib.pyplot as plt

ks_matrix = results_df.pivot(index="Fold", columns="Asset", values="KS_Distance")
plt.figure(figsize=(12, 6))
sns.heatmap(ks_matrix, annot=True, fmt=".3f", cmap="coolwarm")
plt.title("KS Distance between Real and Synthetic Prices (per Fold/Asset)")
plt.ylabel("Fold")
plt.xlabel("Asset")
plt.tight_layout()
plt.savefig("ks_distance_heatmap.png")
plt.show()





# 🔮 Generate Synthetic Forecast
LAST_KNOWN_PRICES = np.array([
    15.136, 20.3352, 17.563, 8.6973, 1.1579, 1.3435,
    20.716, 141.5, 164.252, 9.95, 30.14, 2004.0
])




G.eval()
synthetic_df = generate_synthetic_forecast(G)
display(synthetic_df.head())


# Extract real returns (e.g., from the validation set)
real_returns_flat = val_seq.flatten()  # make sure this exists from TSCV loop

# Extract synthetic returns
synthetic_returns_flat = returns.flatten()

# Evaluate and display
metrics_df = evaluate_synthetic_vs_real(real_returns_flat, synthetic_returns_flat)
print(metrics_df.to_string(index=False))

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def plot_dimensionality_reduction(real_all, synthetic_all, method="PCA"):
    """
    real_all, synthetic_all: list of arrays, each of length T
    """
    all_series = []
    labels = []

    for i, series in enumerate(real_all):
        all_series.append(series)
        labels.append(f"Real_{i}")
    for i, series in enumerate(synthetic_all):
        all_series.append(series)
        labels.append(f"Synth_{i}")

    X = np.array(all_series)

    if method == "PCA":
        reducer = PCA(n_components=2)
    elif method == "tSNE":
        reducer = TSNE(n_components=2, perplexity=5, learning_rate='auto', init='random', random_state=42)
    else:
        raise ValueError("Method must be 'PCA' or 'tSNE'.")

    X_reduced = reducer.fit_transform(X)

    df_plot = pd.DataFrame({
        "x": X_reduced[:, 0],
        "y": X_reduced[:, 1],
        "Type": ["Real"] * len(real_all) + ["Synthetic"] * len(synthetic_all),
        "Asset": [f"A{i}" for i in range(len(real_all))] * 2
    })

    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df_plot, x="x", y="y", hue="Type", style="Asset", s=100)
    plt.title(f"{method} of Real vs Synthetic Price Trajectories")
    plt.tight_layout()
    plt.savefig(f"{method.lower()}_price_trajectories.png")
    plt.show()

# 📈 Plot PCA and t-SNE
plot_dimensionality_reduction(real_all, synthetic_all, method="PCA")
plot_dimensionality_reduction(real_all, synthetic_all, method="tSNE")



# Plot again using final generator
fixed_noise = torch.randn(1, 100).to(device)
plot_result(G, fixed_noise, image_size=1, num_epoch="final", save_dir=".", n_series=12)

🔁 TSCV for Asset 0
📦 Fold 1 | Train: 300, Val: 100
G #parameters:  51092
D #parameters:  38401
Using device: cpu
epoch: [1/200] iteration: [5/5]	Learning rate: 0.0001
Time 0.134s / 5 iters, (0.027)	Data load 0.015s / 5 iters, (0.003089)
Loss_D = 1.36263847 (ave = 1.36770527)
Loss_G = 0.69383931 (ave = 0.69548810)
Loss_GN = 0.00000000 (ave = 0.00000000)

2025-05-25 20:29:36 -----------------------------------------------------------------------------------------------------------------

G grad l2-norm: 0.9601505570546686, value max: 0.02230174094438553
D grad l2-norm: 0.7047106270076978, value max: 0.5003437995910645
epoch: [2/200] iteration: [5/5]	Learning rate: 0.0001
Time 0.958s / 5 iters, (0.192)	Data load 0.911s / 5 iters, (0.182154)
Loss_D = 1.34052205 (ave = 1.35142019)
Loss_G = 0.69135189 (ave = 0.69264959)
Loss_GN = 0.00000000 (ave = 0.00000000)

2025-05-25 20:29:37 -----------------------------------------------------------------------------------------------------------------

ModuleNotFoundError: No module named 'seaborn'